__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
avito_path = 'sevastopol/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-13 13:49:19 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-13 13:49:19 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-13 13:49:21 | AdsListParser: downloader init
[INFO    ] 2022-09-13 13:49:21 | AvitoParser: init
[INFO    ] 2022-09-13 13:49:27 | AvitoParser: 76 pages for read
[INFO    ] 2022-09-13 13:49:27 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-13 13:49:33 | AdsListParser: read page 1
[INFO    ] 2022-09-13 13:49:37 | AdsListParser: read page 2
[INFO    ] 2022-09-13 13:49:42 | AdsListParser: read page 3
[INFO    ] 2022-09-13 13:49:46 | AdsListParser: read page 4
[INFO    ] 2022-09-13 13:49:51 | AdsListParser: read page 5
[INFO    ] 2022-09-13 13:49:57 | AdsListParser: read page 6
[INFO    ] 2022-09-13 13:50:04 | AdsListParser: read page 7
[INFO    ] 2022-09-13 13:50:08 | AdsListParser: read page 8
[INFO    ] 2022-09-13 13:50:13 | AdsListParser: read page 9
[INFO    ] 2022-09-13 13:50:18 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3753
3413


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
534,2489621730,"Объявление «1-к. квартира, 33 м², 4/5 эт.» 14 фотографий",6000000,,"ул. Александра Шостака, 1","Однокомнатная квартира 33 м2, на 4/5 этаже дома. Расположена в г. Севастополь, Гагаринский район, ул. Александра Шостака, д. 1. \n\nКвартира будет идеальным решением для большой семьи. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехнические характеристики: \n",12,2022-09-13 13:55:03.078825
653,2441833498,"Объявление «2-к. апартаменты, 45 м², 1/1 эт.» 18 фотографий",7500000,,"с. Орловка, Качинское ш., 33Б","Мечтаете о собственном уголке на берегу моря? А может Вы ищете готовый бизнес? Эти апартаменты то, что подойдет именно Вам! \n\nПросторные апартаменты в семейном комплексе «Звездный Берег». На территории в 9 гектар, которая утопает в зелени есть все необходи",15,2022-09-13 13:55:03.078825
